# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [14]:
ed = Website("https://edwarddonner.com")
#ed = Website("https://mugenzi93.github.io/index.html")
print(ed.get_contents())
ed.links

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers a

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [15]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [16]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [17]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [18]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [ ]:
# function to get relevant links from a website using OpenAI's API
# Pay attention to how 
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..
# All possible links from Hugging Face

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/reachy-mini',
 '/spaces',
 '/models',
 '/THUDM/GLM-4.1V-9B-Thinking',
 '/HuggingFaceTB/SmolLM3-3B',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/moonshotai/Kimi-K2-Instruct',
 '/apple/DiffuCoder-7B-cpGRPO',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/kontext-community/kontext-relight',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/ilcve21/Sparc3D',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/hackaprompt/Pliny_HackAPrompt_Dataset',
 '/datasets/marcelbinz/Psych-101',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/cais/hle',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/gram

In [ ]:
# Relevant links from Hugging Face
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [23]:
# this function will get all the details of a website, including its links and their contents
# It will return a string with the landing page contents and the contents of each relevant link
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [24]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact page', 'url': 'https://huggingface.co/chat'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Reachy Mini: The Open Robot for AI Builders
Welcome Cohere on the Hub 🔥
Welcome Hyperbolic, Nebius AI Studio, and Novita on the Hub

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [34]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [27]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact page', 'url': 'https://huggingface.co/chat'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nReachy Mini: The Open Robot for AI Builders\nWelcome Cohere on the Hub 🔥\nWelcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTHUDM/GLM-4.1V-9B-Thinking\nUpdated\n3 days ago\n•\n24.3k\n•\n551\nHuggingFaceTB/SmolLM3-3B\nUpdated\n1 day ago\n•\n17k\n•\n330\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n14 days ago\n•\n217k\n•\n1.55k\nmoonshotai/Kimi-K2-Instruct\nUpdated\nabou

In [35]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [29]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face

**Making AI Accessible for Everyone.**  
Hugging Face is a vibrant AI community dedicated to shaping the future of artificial intelligence through collaboration and innovation. Our platform serves as a hub for machine learning practitioners to build, share, and explore models, datasets, and applications, fostering an environment where AI knowledge is accessible to all.

---

## Our Offerings

### Explore Innovative Models
With over 1 million models available, Hugging Face is the home of state-of-the-art models ready for both research and production use. From natural language processing to computer vision, discover cutting-edge models constantly updated and refined by our global community.

### Datasets Galore
Access and contribute to a vast library of over 250,000 datasets tailored for every ML task imaginable. Our community-driven approach ensures that you have a wealth of resources at your fingertips, powering your projects with the latest in AI research.

### Community Spaces
Collaborate through our interactive spaces — from generating applications with DeepSite v2 to exploring dynamic image-to-video transformations. Join a community of creators and become part of the collaborative spirit of Hugging Face.

### Enterprise Solutions
Our enterprise offerings empower organizations with advanced tools to develop AI solutions. Benefit from enterprise-grade security, dedicated support, and optimized compute solutions that accelerate your ML initiatives starting at competitive pricing.

---

## Our Culture

At Hugging Face, we believe in the power of community. Our mission revolves around transparency, collaboration, and inclusiveness. We encourage open-source contributions and celebrate every innovation in machine learning, making it a friendly environment for both seasoned experts and newcomers.

- **Collaboration:** Connect with other researchers and developers to build and share.
- **Transparency:** Open-source philosophy at the core of everything we do.
- **Inclusiveness:** A welcoming atmosphere for everyone, from hobbyists to industry leaders.

---

## Join Us

### Career Opportunities
Grow your career with Hugging Face as we seek talented individuals passionate about AI and technology. Join our diverse team of innovators and work on transformative projects in the machine learning landscape. Explore current job openings and become a part of our mission to democratize AI.

**Explore Opportunities → [Careers Page]**

---

## Our Customers

Join over **50,000 organizations** relying on Hugging Face for their AI solutions, including top names like:

- **Microsoft**
- **Google**
- **Amazon**
- **Meta**

These industry leaders trust Hugging Face to power their AI initiatives with cutting-edge technology and reliable infrastructure.

---

## Get Started Today!

Whether you're an AI enthusiast, a seasoned developer, or a business looking to innovate, there is a place for you at Hugging Face. 

- **Create**: Start building with our models and datasets today.
- **Collaborate**: Join our community to share knowledge and innovate together.
- **Transform**: Use our tools to craft the next generation of AI solutions.

**Join the AI revolution at Hugging Face!**  
[Explore Now](https://huggingface.co)

--- 

For more details, resources, and updates, connect with us on social media platforms: [GitHub](#), [Twitter](#), [LinkedIn](#), [Discord](#).

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [30]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [31]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

### **Company Overview**
**Hugging Face** is an innovative AI community dedicated to building the future of machine learning. Our platform serves as a collaborative hub for developers, researchers, and enthusiasts to share and explore cutting-edge models, datasets, and applications. With a vibrant community of over 50,000 organizations and industry leaders such as Google, Microsoft, and Amazon, we are at the forefront of AI advancement.

---

### **Our Offerings**
- **Models:** Access and contribute to over 1 million machine learning models, including state-of-the-art solutions for text, image, video, audio, and 3D.
- **Datasets:** Share and explore 250k+ datasets tailored for various machine learning tasks.
- **Spaces:** Host and collaborate on unlimited public models, datasets, and applications.
- **Enterprise Solutions:** Our paid Compute and Enterprise offerings provide advanced security features, dedicated support, and built-in access controls for teams of any size.

---

### **Community & Culture**
At Hugging Face, we believe in the power of collaboration. Our vibrant community fosters an environment of openness and innovation, where contributors not only showcase their work but also help others grow. Our commitment to open-source principles guides us in building tools and technologies that empower AI builders globally.

---

### **Join Our Team**
We are on the lookout for passionate individuals ready to make an impact in the AI field. Careers at Hugging Face encompass various areas such as engineering, marketing, and community management, all reflecting our core values of collaboration, innovation, and inclusivity.

**Why work with us?**
- **Empowerment:** You will have the freedom to explore your ideas and drive projects that matter.
- **Collaboration:** Immerse yourself in a community that values your contributions and supports growth.
- **Innovation:** Be part of a team that is consistently pushing the boundaries of what’s possible in AI. 

---

### **Connect with Us**
Explore our offerings on our website, and join our community today! Together, we can shape the future of AI.

- **Website:** [Hugging Face](https://huggingface.co)
- **Social Media:** [GitHub](https://github.com/huggingface) | [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://www.linkedin.com/company/huggingface) | [Discord](https://discord.gg/huggingface)

---

**Hugging Face – Building the Future Together!**

In [ ]:
# Try changing the website to Kaggle:

stream_brochure("Kaggle", "https://kaggle.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://kaggle.com/about'}, {'type': 'careers page', 'url': 'https://kaggle.com/careers'}, {'type': 'company page', 'url': 'https://kaggle.com/company'}]}


# Kaggle Brochure

## Welcome to Kaggle

**Kaggle: Your Machine Learning and Data Science Community**  
At Kaggle, we believe in making data science accessible for everyone. Our platform connects a passionate community of data scientists and machine learning enthusiasts who collaborate, compete, and learn from one another.

---

## About Us

### Kaggle: Your Home for Data Science
Kaggle is dedicated to providing a rich environment for data enthusiasts. Our mission is to empower individuals and organizations to leverage data science to solve real-world problems. We host competitions, datasets, and kernels (code notebooks) that allow users to showcase their skills and improve their knowledge.

---

## Company Culture

At Kaggle, we foster a collaborative and inclusive workplace environment. Our culture encourages innovation, learning, and sharing knowledge. We celebrate diversity and strive to create an atmosphere where everyone can thrive. Our team is made up of individuals who are not only experts in their fields but are also passionate about mentoring and guiding others in their data science journey.

---

## Customers

Our users range from individual data scientists and students to large organizations seeking to harness the power of data. Kaggle is used by over 10 million data enthusiasts globally, making it one of the largest platforms for machine learning and AI collaboration.

---

## Careers at Kaggle

### Work with Us!
Join our dynamic team where you can make a real impact in the data science community. We offer a variety of career opportunities in areas such as machine learning, data engineering, product management, and more. Our employees enjoy a flexible work environment, competitive salaries, and opportunities for continuous learning and professional development.

If you are passionate about data science and want to help shape the future of analytics, explore open positions on our careers page and become a part of our innovative team!

---

## Get Involved

Whether you are a seasoned data scientist or just starting your journey, Kaggle offers a multitude of resources and opportunities to grow your skills and connect with others in the field. Join our community today and start exploring the world of data!

---

For more information, visit our website at [Kaggle](https://www.kaggle.com) and become a part of our thriving data science community!

In [36]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Hugging Face", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}



# Hugging Face: Embrace the Future with a Hug!

## 🤗 Welcome to Hugging Face!

Are you tired of machines that just sit there, not giving you a comforting embrace? Look no further! At Hugging Face, we’re not just an AI community; we’re the *AI community*—building the future one model and dataset at a time. If you ever felt lonely in your coding endeavours, we promise you’ll feel cozy here!

---

## 🌍 Who’s Using Our Magic?

Why stop at just being a “successful” startup when you can have over **50,000 organizations** on your friend list? From **Meta** to **Amazon**, and even **Microsoft** (who knew they loved hugging, too?), our models are so popular they should have their own fan clubs! 

---

## 💡 What Do We Offer?

- **1 Million+ Models**: Yes, you heard that right. Our collection makes the local library look like a bookshop!
- **Datasets Galore**: Track the latest datasets because who wants to work with stale data?
- **Spaces for All**: Running apps that even your grandma could manage… if she knows how to push buttons!
- **Epic Compute & Enterprise Solutions**: Because scaling up your AI shouldn’t feel like you’re trying to lift Thor's hammer!

---

## 🌈 Hugging Face Culture: More than Just Tech

Our team features visionaries and dreamers, but mostly people who genuinely enjoy coding and chatting about AI like it’s the latest gossip. Expect:
- **Coffee & Collaboration**: Our secret to success? Caffeine and great conversations about data!
- **Open Source Love**: Because sharing is caring, even in the world of AI.
- **Work-from-Anywhere Karma**: Define your workspace, even if it’s from your favorite coffee shop or that cozy blanket fort.

---

## 🚀 Careers: Join the Hugging Family!

We’re always looking for talented, fun individuals who are ready to embrace the world of AI. Interested? Here’s a quick checklist on how to belong:
- Must know more Python than a pet snake.
- Ability to cheer when any AI model updates.
- Someone who thinks datasets are sexy (yes, we said it!).

**You don’t just join a company; you join a community of AI Avengers!** 🦸‍♂️🦸‍♀️

---

## 🥳 Ready to Get Started?

Visit our website to sign up for the future! You won’t just be a part of a community; you’ll feel like you’ve joined a caring family that hugs you with knowledge and support.

**Touchdowns for average Joe? More like AI love for the LAI-n!** 🎉

Hug it out with us. Let's build something beautiful together at Hugging Face!

---

🤗 **Hugging Face - Where AI Meets Community and Fun!**



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>